In [3]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '../')
import script as sc
import tensorflow as tf

/Users/yashmehra/Library/Python/3.6/lib/python/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
tf.logging.set_verbosity(tf.logging.INFO)

# Data Import

In [7]:
# data = pd.read_csv(r"C:\Users\yashm\Google Drive\Data Capstone_\Project Folder\PreWideData\cnn_wide.csv")
# data = pd.read_csv(sc.load_Data('AGG-Yash', 'cnn_wide.csv'))
data = pd.read_csv("/Users/yashmehra/Google Drive/Data Capstone /Project Folder/PreWideData/cnn_wide.csv")
data.shape

(116090, 243)

In [8]:
data[np.arange(240,480).astype(str)] = data[np.arange(240).astype(str)].shift(-1)
data[['time','activity','labels']] = data[['time','activity','labels']].shift(-1)
data = data.drop(data.index[len(data) - 1])

labels = data['activity']
K = len(data['activity'].unique())
features = data[np.arange(480).astype(str)]

In [15]:
data.shape, features.shape, K, len(labels)

((116089, 483), (116089, 480), 2, 116089)

In [37]:
labels = data['activity']
K = len(data['activity'].unique())
features = data[data.columns[1:-2]]

In [38]:
features.shape, len(labels), K

((116090, 240), 116090, 2)

In [17]:
# This is a sanity check to make sure TensorFlow is reshaping the features 
# into the 3 channels (x, y, z) correctly.
graph = tf.Graph()
with graph.as_default():
    train_features = tf.placeholder(tf.float32, shape=(None, 480))
    x = tf.reshape(train_features, [-1, 160, 3])
    
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print(sess.run(x, feed_dict={train_features: features}))

[[[-0.605  0.617  0.07 ]
  [-0.621  0.602  0.023]
  [-0.605  0.59  -0.016]
  ...
  [-0.688  0.723  0.613]
  [-0.68   0.727  0.598]
  [-0.68   0.719  0.586]]

 [[-0.512  0.914 -0.191]
  [-0.441  0.918 -0.152]
  [-0.391  0.914 -0.102]
  ...
  [-0.84   0.359  0.473]
  [-0.867  0.359  0.434]
  [-0.895  0.363  0.422]]

 [[-0.672  0.711  0.586]
  [-0.648  0.684  0.582]
  [-0.621  0.668  0.574]
  ...
  [-0.895  0.402  0.465]
  [-0.875  0.398  0.445]
  [-0.859  0.391  0.441]]

 ...

 [[ 0.762  0.258 -0.625]
  [ 0.762  0.258 -0.625]
  [ 0.762  0.258 -0.625]
  ...
  [ 0.762  0.258 -0.625]
  [ 0.762  0.258 -0.625]
  [ 0.762  0.258 -0.625]]

 [[ 0.762  0.258 -0.625]
  [ 0.762  0.258 -0.625]
  [ 0.762  0.258 -0.625]
  ...
  [ 0.762  0.258 -0.625]
  [ 0.762  0.258 -0.625]
  [ 0.762  0.258 -0.625]]

 [[ 0.762  0.258 -0.625]
  [ 0.762  0.258 -0.625]
  [ 0.762  0.258 -0.625]
  ...
  [ 0.762  0.258 -0.625]
  [ 0.762  0.258 -0.625]
  [ 0.762  0.258 -0.625]]]


# Hyper Parameter Tuning

In [18]:
num_epochs = 200
learning_rate = 0.08

In [22]:
graph = tf.Graph()
with graph.as_default():
    # Placeholders for the actual data.
    train_features = tf.placeholder(tf.float32, shape=(None, 480))
    train_labels = tf.placeholder(tf.int32, shape=(None, ))
    
    # Reshaping the data
    x = tf.reshape(train_features, [-1, 160, 3])
    y = tf.one_hot(
        train_labels,
        depth=K
    )
    
    # Convolution 1
    conv1 = tf.layers.conv1d(inputs=x, 
                             filters=8, 
                             kernel_size=8, 
                             padding="same", 
                             data_format="channels_last",
                             activation=tf.nn.relu)
    # Max Pooling 1 (reduces samples from 80 --> 31)
    pool1 = tf.layers.max_pooling1d(inputs=conv1,
                                    pool_size=40, 
                                    strides=2,
                                    data_format="channels_last")
    
    # Convolution 2
    conv2 = tf.layers.conv1d(inputs=pool1, 
                             filters=16, 
                             kernel_size=8, 
                             padding="same", 
                             data_format="channels_last",
                             activation=tf.nn.relu)
    # Max Pooling 2 (reduces samples from 39 --> 17)
    pool2 = tf.layers.max_pooling1d(inputs=conv2,
                                    pool_size=3, 
                                    strides=2,
                                    data_format="channels_last")
    
    # Flatten the Pooled Data
    pool2_flat = tf.reshape(pool2, [-1, 30 * 16])
    
    # Dense Layer (try adding dropout?)
    dense = tf.layers.dense(inputs=pool2_flat, units=128, activation=tf.nn.relu)
    
    # Logits Layer
    logits = tf.layers.dense(inputs=dense, units=K)
    
    # Define loss function and optimizer
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            logits=logits, 
            labels=y
        )
    )
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    # Define accuracy (so that it can be calculated and printed)
    accuracy = tf.reduce_mean(
        tf.cast(
            tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1)),
            tf.float32
        )
    )
    
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    i = 0
    prev = 0
    for _ in range(num_epochs):
        feed_dict = {
            train_features: features,
            train_labels: labels
        }
        _, _, acc = sess.run(
            [optimizer, loss, accuracy], 
            feed_dict=feed_dict)
        print("Epoch: ", i, " Acc: ", acc*100, " Change: ", (acc-prev)*100)
        i = i + 1
        prev = acc

Epoch:  0  Acc:  44.59337294101715  Change:  44.59337294101715


KeyboardInterrupt: 